In [1]:
import json
import re
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from time import gmtime, strftime

def now():
    """
    Returns
    -------
    Current time : str
        eg: 2018-11-22 13:35:23
    """
    return strftime("%Y-%m-%d %H:%M:%S", gmtime())

def get_soup(url, headers=None):
    """
    Arguments
    ---------
    url : str
        Web page url
    headers : dict
        Headers for requests. If None, use Mozilla/5.0 as default user-agent
    Returns
    -------
    soup : bs4.BeautifulSoup
        Soup format web page
    """

    if headers is None:
        headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'}
    r = requests.get(url, headers=headers)
    html = r.text
    page = BeautifulSoup(html, 'lxml')
    return page

doublespace_pattern = re.compile('\s+')
lineseparator_pattern = re.compile('\n+')

def normalize_text(text):
    doublespace_pattern = re.compile('\s+')
    lineseparator_pattern = re.compile('\n+')
    text = text.replace('\t', ' ')
    text = text.replace('\r', ' ')
    text = lineseparator_pattern.sub('\n', text)
    text = doublespace_pattern.sub(' ', text)
    return text.strip()

In [2]:
from nltk.tokenize import sent_tokenize
from dateutil.parser import parse
import re

## this function need to url from urls. use for

def parse_article(url):
    def parse_title(soup):
        title = soup.find('div', class_='view_headline HD')
        if not title:
            return 'title error'
        return title.text
    def parse_date(soup):
        date = soup.find_all('div', class_ = 'view_date')[0].text[9:19] ## need to fix date part by regex 
        if not date:
            return 'date error'
        return parse(date).strftime("%Y-%m-%d")
    
    def parse_content(soup):
        content = soup.find('div', itemprop='articleBody')
        return normalize_text(content.text)

    soup = get_soup(url)

    fist_dic =  {
            'url': url,
            'title': parse_title(soup),
            'subtitle' : "",
            'date': parse_date(soup),
            'content' :parse_content(soup),
            'category' : url.split("/")[4],
            'source': 'koreatimes',
            'scrap_time': now()
    }
    return fist_dic

In [8]:
import json
import time
import requests
import re
from dateutil.parser import parse

## caution: global times limit page 100, you should use date 


def yield_latest_article(begin_date,end_date,  max_num=10, sleep=1.0):
    """
    Artuments
    ---------
    begin_date : str
        eg. 20180701
    end_date :str
        eg. 20190331
    max_num : int
        Maximum number of news to be scraped
    sleep : float
        Sleep time. Default 1.0 sec

    It yields
    ---------
    news : json object
    """

    # prepare parameters

    for page in range(1, 10000):
        # get urls
        page = str(page)
        url = "https://www.koreatimes.co.kr/www2/common/search.asp?kwd=&pageNum={}&pageSize=10&category=total&sort=&startDate={}&endDate={}&date=0&srchFd=&range=&author=all&authorData=&mysrchFd=".format(page, begin_date, end_date)
        soup = get_soup(url)
        sub_links = soup.find('tbody', id = 'divSearchList').find_all('a')
        links = [a['href'] for a in sub_links]
        print(links)
        for a in links:
            try:
                print(a)
                news_json = parse_article(a)
                print(news_json)
                time.sleep(sleep)
            except:
                print("this "+url+" is not working")
                continue

In [ ]:
 yield_latest_article(20210901, 20210901)

['/www/sublist_740.html', '/www/sublist_745.html', 'http://www.koreatimes.co.kr/www/nation/2021/09/113_314896.html', 'http://www.koreatimes.co.kr/www/nation/2021/09/113_314896.html', '/www/nation/2021/09/745_314896.html', '/www/sublist_740.html', '/www/sublist_742.html', 'http://www.koreatimes.co.kr/www/nation/2021/09/113_314895.html', 'http://www.koreatimes.co.kr/www/nation/2021/09/113_314895.html', '/www/nation/2021/09/742_314895.html', '/www/sublist_103.html', 'http://www.koreatimes.co.kr/www/nation/2021/09/103_314892.html', 'http://www.koreatimes.co.kr/www/nation/2021/09/103_314892.html', '/www/nation/2021/09/103_314892.html', '/www/sublist_501.html', '/www/sublist_683.html', 'http://www.koreatimes.co.kr/www/world/2021/09/501_314891.html', 'http://www.koreatimes.co.kr/www/world/2021/09/501_314891.html', '/www/world/2021/09/683_314891.html', '/www/sublist_113.html', '/www/sublist_120.html', 'http://www.koreatimes.co.kr/www/nation/2021/09/113_314890.html', 'http://www.koreatimes.co.k

http://www.koreatimes.co.kr/www/nation/2021/09/113_314896.html
http://www.koreatimes.co.kr/www/nation/2021/09/113_314896.html
{'url': 'http://www.koreatimes.co.kr/www/nation/2021/09/113_314896.html', 'title': 'JeongJin-woon to return to big screen as rookie cop in crime action film ', 'subtitle': '', 'date': '2021-09-01', 'content': 'Jeong Jin-woon in a scene from the film, "Brother" / Courtesy of The Great Company Singer-actor JeongJin-woon will return to the big screen as a rookie cop in the crime action film, "Brother," three months after starring in his first full-length horror film, "Only I Can See." 가수 겸 배우 정진운이 첫 장편 공포영화 ‘나만보이니\' 주연으로 출연한지 3개월 만에 범죄 액션영화 ‘브라더\'의 신입 형사로 스크린에 복귀한다. In the upcoming film, Jeong will play the role of Kang-soo, a righteous cop who unexpectedly joins hands with gang boss Yong-sik (Jo Jae-yun). Director Shin Keun-ho, well-known for his previous film, "Romantic Debtors," (2010) directed the flick. 이번 작품에서 정진운은 범죄조직 행동대장 용식(조재윤)과 뜻밖에 손을 잡는 의로운 형사 강수역을 맡게된

{'url': 'http://www.koreatimes.co.kr/www/nation/2021/09/113_314895.html', 'title': 'Jincheon praised for housing Afghan evacuees ', 'subtitle': '', 'date': '2021-09-01', 'content': "A notice announcing the temporary suspension of operation is posted, Monday, on Jincheon County's online shopping mall, which sells regional specialties such as rice. Screen capture from JC mallJincheon County located in North Chungcheong Province is receiving applause from people across the country for housing hundreds of Afghans who were evacuated from their homeland, which has been overrun by the Taliban.충청북도 진천군이 탈레반에 점령된 아프가니스탄에서 탈출한 수백 명의 아프간인을 수용해 전 국민에게 찬사를 받았다. As a means to express their respect for the county's acceptance of the Afghan evacuees, many people have been flocking to its online shopping mall, which sells regional specialties such as rice, in recent days. A flood of orders from all over the country has resulted in the temporary suspension of the operation of the online mall.진천군의 아프간 난민

{'url': 'http://www.koreatimes.co.kr/www/nation/2021/09/103_314892.html', 'title': 'North Korea turns down Sinovac COVID-19 vaccine doses: WSJ ', 'subtitle': '', 'date': '2021-09-01', 'content': 'A nurse handles a package of refrigerated COVID-19 vaccine doses from Chinese company Sinovac, in Hangzhou in east China\'s Zhejiang province in this March 15 file photo. AP-YonhapNorth Korea has rejected roughly three million COVID-19 vaccine doses developed by China\'s Sinovac Biotech, saying the shots should be sent to harder-hit countries, the Wall Street Journal reported on Wednesday, citing UNICEF.The agency, which helps deliver COVID-19 shots on behalf of the COVAX scheme, told WSJ that North Korea\'s public health ministry declined the shipment, citing the limited global supply for vaccines and continuing virus surges elsewhere.North Korea had in July rejected shipments of AstraZeneca\'s COVID-19 vaccine due to concerns over side effects, according to a South Korean think-tank.The recl

{'url': 'http://www.koreatimes.co.kr/www/world/2021/09/501_314891.html', 'title': 'Pope defends deal with China, says dialogue necessary ', 'subtitle': '', 'date': '2021-09-01', 'content': 'Pope Francis attends his weekly general audience, held in the Paul VI hall, at the Vatican, Wednesday, Wednesday. AP-YonhapPope Francis has defended the Vatican deal with China\'s communist government on the appointment of Catholic bishops, saying an uneasy dialogue is better than no dialogue at all.In an interview with Spanish radio network COPE broadcast on Wednesday, Francis compared the Vatican\'s dialogue with China to those with East European countries during the Cold War which eventually resulted in many freedoms for the Church there."China is not easy, but I am convinced that we should not give up dialogue," he said in some of his most comprehensive comments to date on the issue.Last September, the Vatican renewed a 2018 accord with Beijing that gives the pope final say over the appointment 

...

/www/biz/2021/09/126_314821.html
this https://www.koreatimes.co.kr/www2/common/search.asp?kwd=&pageNum=8&pageSize=10&category=total&sort=&startDate=20210901&endDate=20210901&date=0&srchFd=&range=&author=all&authorData=&mysrchFd= is not working
/www/sublist_113.html
this https://www.koreatimes.co.kr/www2/common/search.asp?kwd=&pageNum=8&pageSize=10&category=total&sort=&startDate=20210901&endDate=20210901&date=0&srchFd=&range=&author=all&authorData=&mysrchFd= is not working
/www/sublist_205.html
this https://www.koreatimes.co.kr/www2/common/search.asp?kwd=&pageNum=8&pageSize=10&category=total&sort=&startDate=20210901&endDate=20210901&date=0&srchFd=&range=&author=all&authorData=&mysrchFd= is not working
http://www.koreatimes.co.kr/www/nation/2021/09/113_314820.html
/www/biz/2021/09/126_314821.html
this https://www.koreatimes.co.kr/www2/common/search.asp?kwd=&pageNum=8&pageSize=10&category=total&sort=&startDate=20210901&endDate=20210901&date=0&srchFd=&range=&author=all&authorData=&mysrchFd

{'url': 'http://www.koreatimes.co.kr/www/nation/2021/09/113_314820.html', 'title': '402 female NCOs join ROK Army', 'subtitle': '', 'date': '2021-09-01', 'content': "Non-commissioned officers newly joining the Republic of Korea Army for the period of the second half of 2021 attend their induction ceremony held at the Korea Army Non-Commissioned Officer Academy in Iksan, North Jeolla Province, Aug. 27. Courtesy of Republic of Korea Army By Jung Da-minA total of 487 non-commissioned officers (NCOs) have newly enlisted in the Republic of Korea Army through the Army's induction ceremony for the second half of 2021, held at the Korea Army Non-Commissioned Officer Academy in Iksan, North Jeolla Province, Aug. 27. Among them, 402 were female members, accounting for about 82.5 percent of the total. The Army said it was the largest number of female NCOs enlisting at a single induction ceremony, adding that it was a result of the military's Defense Reform 2.0, a comprehensive reform initiative a

http://www.koreatimes.co.kr/www/art/2021/09/398_314796.html
http://www.koreatimes.co.kr/www/art/2021/09/398_314796.html
{'url': 'http://www.koreatimes.co.kr/www/art/2021/09/398_314796.html', 'title': 'Musicals to be adapted to TV series, films ', 'subtitle': '', 'date': '2021-09-01', 'content': 'A scene from the Korean musical "Cha_me" / Courtesy of PAGE1By Park Ji-wonWeb and more web comics, films and dramas are being adapted into other content formats. For example, Netflix\'s "Sweet Home," one of the most popular series in the streaming service last year, is a drama adaptation of a web comic of the same title. Although they have not been launched officially, some companies started projects to adapt TV series into web comics including "My Mister."Unlike these genres, musicals have never been adapted into other genres, partly because there were not enough original musicals that can be made into other content formats. But a growing number of production companies are setting their sights

this https://www.koreatimes.co.kr/www2/common/search.asp?kwd=&pageNum=8&pageSize=10&category=total&sort=&startDate=20210901&endDate=20210901&date=0&srchFd=&range=&author=all&authorData=&mysrchFd= is not working
http://www.koreatimes.co.kr/www/nation/2021/09/653_307268.html
this https://www.koreatimes.co.kr/www2/common/search.asp?kwd=&pageNum=8&pageSize=10&category=total&sort=&startDate=20210901&endDate=20210901&date=0&srchFd=&range=&author=all&authorData=&mysrchFd= is not working
http://www.koreatimes.co.kr/www/nation/2021/09/653_307268.html
this https://www.koreatimes.co.kr/www2/common/search.asp?kwd=&pageNum=8&pageSize=10&category=total&sort=&startDate=20210901&endDate=20210901&date=0&srchFd=&range=&author=all&authorData=&mysrchFd= is not working
/www/nation/2021/09/653_307268.html
this https://www.koreatimes.co.kr/www2/common/search.asp?kwd=&pageNum=8&pageSize=10&category=total&sort=&startDate=20210901&endDate=20210901&date=0&srchFd=&range=&author=all&authorData=&mysrchFd= is not wo

KeyboardInterrupt: 